In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    # ("DF", [0.1]),
    # ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA" and attack_name!= "AutoPGD" and attack_name!= "DF":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
# Collect 20% of eps = 0.1 data
from sklearn.model_selection import train_test_split

_, X_20, _ , y_20 = train_test_split(x_test_adv_0_1, y_label_0_1, test_size=0.2, stratify=y_label_0_1, random_state=42)


In [6]:
X_20.shape

(136766, 56)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb20 = XGBClassifier()
xgb20.fit(X_20, y_20)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf20 = RandomForestClassifier()
rf20.fit(X_20, y_20)

print("DT")
from sklearn.tree import DecisionTreeClassifier
dt20 = DecisionTreeClassifier()
dt20.fit(X_20, y_20)

xgb
RF
DT


DecisionTreeClassifier()

In [8]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = xgb20.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_Exclude3Attack/UNSW_Input20/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_Exclude3Attack/UNSW_Input20/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [   214    197   1839   2138     29    442    110 108766    206     30]
(214, 56) (214,)
Save 0 to baseline_Def1.npy
(197, 56) (197,)
Save 1 to baseline_Def2.npy
(1839, 56) (1839,)
Save 2 to baseline_Def3.npy
(2138, 56) (2138,)
Save 3 to baseline_Def4.npy
(29, 56) (29,)
Save 4 to baseline_Def5.npy
(442, 56) (442,)
Save 5 to baseline_Def6.npy
(110, 56) (110,)
Save 7 to baseline_Def8.npy
(108766, 56) (108766,)
Save 8 to baseline_Def9.npy
(206, 56) (206,)
Save 9 to baseline_Def10.npy
(30, 56) (30,)
Save 10 to baseline_Def11.npy
Execution Time: 0.247228 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [218950    488   1985   3849    504   5046     74 110923     13     81]
(218950, 56) (218950,)
Save 0 to BIM_Def1.npy
(488, 56) (488,)
Save 1 to BIM_Def2.npy
(1985, 56) (1985,)
Save 2 to BIM_Def3.npy
(3849, 56) (3849,)
Save 3 to BIM_Def4.npy
(504, 56) (504,)
Save 4 to BIM_Def5.npy
(5046, 56) (5046,)
Save 5 to BIM_Def6.npy
(74, 56) (74,)
Save 7 to BIM

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf20.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude3Attack/UNSW_Input20/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Exclude3Attack/UNSW_Input20/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [   258    144   1647   1807     24    519     84 109306    153     29]
(258, 56) (258,)
Save 0 to baseline_Def1.npy
(144, 56) (144,)
Save 1 to baseline_Def2.npy
(1647, 56) (1647,)
Save 2 to baseline_Def3.npy
(1807, 56) (1807,)
Save 3 to baseline_Def4.npy
(24, 56) (24,)
Save 4 to baseline_Def5.npy
(519, 56) (519,)
Save 5 to baseline_Def6.npy
(84, 56) (84,)
Save 7 to baseline_Def8.npy
(109306, 56) (109306,)
Save 8 to baseline_Def9.npy
(153, 56) (153,)
Save 9 to baseline_Def10.npy
(29, 56) (29,)
Save 10 to baseline_Def11.npy
Execution Time: 1.077671 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [216457    418   2003   2718     85   5109      1     87 115006     13
     16]
(216457, 56) (216457,)
Save 0 to BIM_Def1.npy
(418, 56) (418,)
Save 1 to BIM_Def2.npy
(2003, 56) (2003,)
Save 2 to BIM_Def3.npy
(2718, 56) (2718,)
Save 3 to BIM_Def4.npy
(85, 56) (85,)
Save 4 to BIM_Def5.npy
(5109, 56) (5109,)
Save 5 to BIM_Def6.npy
(1, 56) (1,)
Save 6 t

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = dt20.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_Exclude3Attack/UNSW_Input20/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_Exclude3Attack/UNSW_Input20/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [   577    535   2540   2189     73   1349    347 105947    265    149]
(577, 56) (577,)
Save 0 to baseline_Def1.npy
(535, 56) (535,)
Save 1 to baseline_Def2.npy
(2540, 56) (2540,)
Save 2 to baseline_Def3.npy
(2189, 56) (2189,)
Save 3 to baseline_Def4.npy
(73, 56) (73,)
Save 4 to baseline_Def5.npy
(1349, 56) (1349,)
Save 5 to baseline_Def6.npy
(347, 56) (347,)
Save 7 to baseline_Def8.npy
(105947, 56) (105947,)
Save 8 to baseline_Def9.npy
(265, 56) (265,)
Save 9 to baseline_Def10.npy
(149, 56) (149,)
Save 10 to baseline_Def11.npy
Execution Time: 0.070720 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [185076   2186   9167  10584  24382   8332     16    270  98859   2603
    438]
(185076, 56) (185076,)
Save 0 to BIM_Def1.npy
(2186, 56) (2186,)
Save 1 to BIM_Def2.npy
(9167, 56) (9167,)
Save 2 to BIM_Def3.npy
(10584, 56) (10584,)
Save 3 to BIM_Def4.npy
(24382, 56) (24382,)
Save 4 to BIM_Def5.npy
(8332, 56) (8332,)
Save 5 to BIM_Def6.npy
(16, 

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}20")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_Exclude3Attack/UNSW_Input20/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input20_Exclude3Attack.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB20,214,197,1839,2138,29,442,0,110,108766,206,30,113971
1,XGB20,218950,488,1985,3849,504,5046,0,74,110923,13,81,341913
2,XGB20,223068,70,1518,5358,489,2335,0,409,108592,2,72,341913
3,XGB20,218950,488,1985,3849,504,5046,0,74,110923,13,81,341913
4,XGB20,309170,59,1337,5913,276,4127,0,23,20937,67,4,341913
5,XGB20,202922,749,4267,7200,493,1909,0,251,123585,452,85,341913
6,XGB20,740,564,5666,5811,87,1194,0,327,326911,525,88,341913
7,XGB20,88922,31,2069,1781,14,156,0,53,248865,7,15,341913
8,XGB20,213622,491,1711,6970,679,2338,0,0,116023,11,68,341913
9,XGB20,218307,123,2044,5615,1162,4428,0,17,110082,16,119,341913
